In [1]:
import numpy as np
import requests as r
import pandas as pd
import json
from bs4 import BeautifulSoup 
import matplotlib.pyplot as plt
import seaborn as sns
import pickle

In [2]:
#read in csv to dataframe
df = pd.read_csv('spreadspoke_scores.csv')
#Eliminate observations that do not have the data we need
df= df[(df.spread_favorite.notnull()) & (df.over_under_line.notnull())& (df.score_home.notnull()) &(df.score_away.notnull())&(df.weather_temperature.notnull())&(df.weather_wind_mph.notnull())]
del df['weather_humidity'], df['stadium_neutral']
#Only want post 1987 data to avoid last NFL strike
df= df[df.schedule_season>1987]
#Fill in weather detail data
#df.weather_detail=df.weather_detail.fillna('CLEAR')- considered this but weather detail column was too unpopulated


In [3]:
## teamSwitch takes a string of the name of any football team from the past 30 years and returns the current abbreviation
def teamSwitch (full):
    abv=''
    if full == 'New York Jets' or full== 'New York Giants':
        for let in range(len(full)):
            if let == 0:
                abv+=full[let]
            elif full[let]==' ':
                abv+=full[let+1]
    elif full.startswith('New ') or full.startswith('Kans') or full[6:9]=='Bay' or full.endswith('49ers'):
        for let in range(len(full)):
            if let == 0:
                abv+=full[let]
            elif full[let]==' ':
                abv+=full[let+1]
                return abv
    elif full.endswith('Oilers'):
        return "TEN"
    elif full.endswith('Cardinals'):
        return 'ARI'
    elif full.endswith('Rams'):
        return 'LAR'
    elif full.endswith('Chargers'):
        return 'LAC'
    elif full.endswith('Raiders'):
        return 'OAK'
    elif full[0:4]== 'Jack':
        return 'JAX'
    else:
        return full[0:3].upper()
    return abv

#weekNums takes the named weeks of the playoff rounds and converts them to ints
def weekNums(cell):
    if cell=='Division':
        cell= int(19)
    elif cell.upper()=='WILDCARD':
        cell=int(18)
    elif cell== 'Conference':
        cell=int(20)
    elif cell.upper()== 'SUPERBOWL':
        cell=int(21)
        
    else:
        cell=int(cell)
    return cell
#takes a pro football stadium and shows its zip code
def zipCode(cell):
    if cell==('Giants Stadium') or cell== ('MetLife Stadium'):
        cell= '07073'
    elif cell=='Lambeau Field':
        cell='54304'
    elif cell=='Arrowhead Stadium':
        cell='64129'
    elif cell== 'Soldier Field':
        cell='60605'
    elif cell==('Louisiana Superdome')or cell== ('Mercedes-Benz Superdome'):
        cell = '70112'
    elif cell=='Qualcomm Stadium':
        cell= '92108'
    elif cell=='Sun Life Stadium'or cell=='Hard Rock Stadium':
        cell='33056'
    elif cell== 'Ralph Wilson Stadium' or cell=='New Era Field':
        cell= '14127'
    elif cell== 'Georgia Dome' or cell=='Atlanta-Fulton County Stadium' :
        cell='30313'
    elif cell == 'Candlestick Park' or cell=='Levi\'s Stadium':
        cell= '95054'
    elif cell == 'University of Phoenix Stadium':
        cell= '85305'
    elif cell=='Oakland Coliseum':
        cell= '94621'
    elif cell== 'Bank of America Stadium':
        cell ='28202'
    elif cell== 'EverBank Field' or cell=='TIAA Bank Field':
        cell='32202'
    elif cell== 'FedEx Field' or cell== 'RFK Memorial Stadium':
        cell='20785'
    elif cell== 'Hubert H. Humphrey Metrodome' or cell=='Mall of America Field' or cell=='U.S. Bank Stadium':
        cell='55414'
    elif cell== 'Edward Jones Dome':  
        cell='63101'
    elif cell=='Texas Stadium' or cell=='Cowboys Stadium' or cell=='AT&T Stadium':
        cell='76011'
    elif cell== 'M&T Bank Stadium':
        cell= '21230'
    elif cell== 'RCA Dome' or cell=='Lucas Oil Stadium':
        cell='46225'
    elif cell== 'Raymond James Stadium' or cell=='Houlihan\'s Stadium':  
        cell='33607'
    elif cell=='FirstEnergy Stadium' or cell=='Cleveland Municipal Stadium':
        cell='44114'  
    elif cell== 'Nissan Stadium'or cell=='LP Stadium':  
        cell='37213'
    elif cell=='Paul Brown Stadium' or cell=='Cinergy Field':
        cell='45202'
    elif cell== 'Heinz Field' or cell=='Three Rivers Stadium':
        cell='15212'
    elif cell== 'Sports Authority Field at Mile High' or cell=='Mile High Stadium':
        cell='80204'
    elif cell== 'Gillette Stadium' or cell=='Foxboro Stadium':  
        cell='02035'
    elif cell=='CenturyLink Field' or cell=='Seattle Kingdome':
        cell='98134'
    elif cell=='Ford Field' or cell=='Pontiac Silverdome':
        cell='48226'  
    elif cell=='Lincoln Financial Field' or cell=='Veterans Stadium':
        cell='19148'
    elif cell=='Los Angeles Memorial Coliseum':
        cell='90037'   
    elif cell=='Anaheim Stadium':
        cell='92806' 
    elif cell=='Sun Devil Stadium':
        cell='85287' 
    elif cell=='Wembley Stadium':
        cell=''
    elif cell=='Husky Stadium':
        cell='98195'
    elif cell== 'TCF Bank Stadium':
        cell= '55455'
    elif cell=='Memorial Stadium (Baltimore)':
        cell='21218'
    elif cell=='StubHub Center':
        cell= '90746'
    elif cell=='Memorial Stadium (Champaign)':
        cell='61820'
    elif cell=='Vanderbilt Stadium':
        cell='37203'
    elif cell=='Rogers Centre':
        cell=''
    elif cell=='Tiger Stadium (LSU)':
        cell='70803'
    elif cell=='Busch Memorial Stadium':
        cell='63102'
    elif cell=='Twickenham Stadium':
        cell=''
    elif cell=='Alamo Dome':
        cell='78203'
    elif cell=='Estadio Azteca':
        cell=''
    elif cell=='Tottenham Stadium':
        cell=''
        
    elif cell==('Reliant Stadium')or cell== ('NRG Stadium')  or cell == ('Houston Astrodome'):
        cell = '77054'
    elif cell==('Memorial Stadium (Clemson)'):
        cell = '29634'
    elif cell==('Liberty Bowl Memorial Stadium'):
        cell = '38104'
        
    return cell


#takes a pro football stadium and return its coordinates
def stadCord(cell):
    if cell==('Giants Stadium') or cell== ('MetLife Stadium'):
        cell= (40.8135 , -74.0745)
    elif cell=='Lambeau Field':
        cell=(44.5013, -88.0622)
    elif cell=='Arrowhead Stadium':
        cell=(39.0489, -94.4839)
    elif cell== 'Soldier Field':
        cell=(41.8623, -87.6167)
    elif cell==('Louisiana Superdome')or cell== ('Mercedes-Benz Superdome'):
        cell = (29.9511, -90.0812)
    elif cell=='Qualcomm Stadium':
        cell= (32.7764, -117.1183)
    elif cell=='Sun Life Stadium'or cell=='Hard Rock Stadium':
        cell=(25.9580, -80.2389)
    elif cell== 'Ralph Wilson Stadium' or cell=='New Era Field':
        cell= (42.7738, -78.7870)
    elif cell== 'Georgia Dome' or cell=='Atlanta-Fulton County Stadium' :
        cell=(33.7577, -84.4008)
    elif cell == 'Candlestick Park' or cell=='Levi\'s Stadium':
        cell= (37.7136, -122.3861)
    elif cell == 'University of Phoenix Stadium':
        cell= (33.5276, -112.2626)
    elif cell=='Oakland Coliseum':
        cell= (37.7516, -122.2005)
    elif cell== 'Bank of America Stadium':
        cell =(35.2258, -80.8528)
    elif cell== 'EverBank Field' or cell=='TIAA Bank Field':
        cell=(30.3238, -81.6375)
    elif cell== 'FedEx Field' or cell== 'RFK Memorial Stadium':
        cell=(38.9077, -76.8644)
    elif cell== 'Hubert H. Humphrey Metrodome' or cell=='Mall of America Field' or cell=='U.S. Bank Stadium':
        cell=(44.9738, -93.2580)
    elif cell== 'Edward Jones Dome':  
        cell=(38.6327, -90.1886)
    elif cell=='Texas Stadium' or cell=='Cowboys Stadium' or cell=='AT&T Stadium':
        cell=(32.84, -96.911)
    elif cell== 'M&T Bank Stadium':
        cell=(39.2780, -76.6227)
    elif cell== 'RCA Dome' or cell=='Lucas Oil Stadium':
        cell=(39.7636, -86.1633)
    elif cell== 'Raymond James Stadium' or cell=='Houlihan\'s Stadium':  
        cell=(27.9758, -82.5033)
    elif cell=='FirstEnergy Stadium' or cell=='Cleveland Municipal Stadium':
        cell=(41.5066, -81.6972)   
    elif cell== 'Nissan Stadium'or cell=='LP Stadium':  
        cell=(36.1663, -86.7713)
    elif cell=='Paul Brown Stadium' or cell=='Cinergy Field':
        cell=(39.095, -84.516)
    elif cell== 'Heinz Field' or cell=='Three Rivers Stadium':
        cell=(40.4466, -80.0158)
    elif cell== 'Sports Authority Field at Mile High' or cell=='Mile High Stadium':
        cell=(39.7438, -105.02)
    elif cell== 'Gillette Stadium' or cell=='Foxboro Stadium':  
        cell=(42.0909, -71.2643)
    elif cell=='CenturyLink Field' or cell=='Seattle Kingdome':
        cell=(47.5952, -122.3316)  
    elif cell=='Ford Field' or cell=='Pontiac Silverdome':
        cell=(42.34, -83.0455)  
    elif cell=='Lincoln Financial Field' or cell=='Veterans Stadium':
        cell=(39.9008, -75.1675) 
    elif cell=='Los Angeles Memorial Coliseum':
        cell=(34.0141, -118.2877)     
    elif cell=='Anaheim Stadium':
        cell=(33.8002, -117.8827)  
    elif cell=='Sun Devil Stadium':
        cell=(33.4263, -111.93)  
    elif cell=='Wembley Stadium':
        cell=(51.5558, -0.2797)
    elif cell=='Husky Stadium':
        cell=(47.6502, -122.3016)
    elif cell== 'TCF Bank Stadium':
        cell= (44.976, -93.225)
    elif cell=='Memorial Stadium (Baltimore)':
        cell=(39.3294, -76.6013)
    elif cell=='StubHub Center':
        cell=(33.864, -118.261)
    elif cell=='Memorial Stadium (Champaign)':
        cell=(40.0991, -88.2358)
    elif cell=='Vanderbilt Stadium':
        cell=(36.1441, -86.8088)
    elif cell=='Rogers Centre':
        cell=(43.6413, -79.3891)
    elif cell=='Tiger Stadium (LSU)':
        cell=(30.4119, -91.1855)
    elif cell=='Busch Memorial Stadium':
        cell=(38.6238, -90.1925)
    elif cell=='Twickenham Stadium':
        cell=(51.4561, -0.3416)
    elif cell=='Alamo Dome':
        cell=(29.4169, -98.4788)
    elif cell=='Estadio Azteca':
        cell=(19.3029, -99.1504)
    elif cell=='Tottenham Stadium':
        cell=(51.6044, -0.0663)
        
    elif cell==('Reliant Stadium')or cell == ('NRG Stadium') or cell == ('Houston Astrodome'):
        cell = (29.6847, -95.4107)
    elif cell==('Memorial Stadium (Clemson)'):
        cell = (34.6788, -82.8432)
    elif cell==('Liberty Bowl Memorial Stadium'):
        cell = (35.1211, -89.9774)
        
    return cell
#changes the date format to be web scrape friendly
def dateSwitch(cell):
    cell= cell[6:]+cell[5]+cell[0:3]+cell[3:5]
    cell=cell.replace('/','-')
    return cell


In [4]:
ppdf= df.copy()
#apply teamSwitch to data
ppdf.team_home=ppdf.team_home.apply(teamSwitch)
ppdf.team_away=ppdf.team_away.apply(teamSwitch)
#create a column to show if the home team is favored
homefave= ppdf.team_home==ppdf.team_favorite_id
ppdf['home_fave']= homefave
#create a column to show the total points scored by both teams
ppdf['total']= ppdf.score_home+ppdf.score_away


#Add stadium coordinates
col=ppdf.copy()
col.stadium=col.stadium.apply(stadCord)
ppdf['coordinates']=col.stadium
col=ppdf.copy()
#Add zipcode
col.stadium=col.stadium.apply(zipCode)
ppdf['zipcode']=col.stadium

#reformat dates
ppdf.schedule_week=ppdf.schedule_week.apply(weekNums)
ppdf=ppdf[ppdf.zipcode!='']

ppdf.schedule_date=ppdf.schedule_date.apply(dateSwitch)


In [5]:
#Create a dict, result, to track how teams do compared to the spread. If favorite wins, the number is negative and  the number is positive if the underdog wins
#Initialize empty dict
result= {}
#loop through dataFrame index to populate dict with keys from the index  and values from columns
for x in ppdf.index:
    if ppdf.loc[x]['home_fave']== True:
        result[x]=ppdf.loc[x]['score_away']-ppdf.loc[x]['score_home']
    else:
        result[x]=ppdf.loc[x]['score_home']-ppdf.loc[x]['score_away']


In [6]:
#create new column for result
result = pd.Series(result)
ppdf['results']=result
ppdf['results-spread']= ppdf['results']-ppdf['spread_favorite']
#using the new results column, create a bolean series to become a new column to indicate if the spread ws covered
cover =ppdf.results<ppdf.spread_favorite
ppdf['favorite_cover']=cover
#create a boolean series to make a column to indicate if the over was reached
ppdf['over_under_line']=ppdf['over_under_line'].astype(float)
over= ppdf['total']>ppdf['over_under_line']
ppdf['over_cover']= over
over_diff= ppdf.over_under_line -ppdf.total
ppdf['over_diff']=over_diff
#rearrange columns for more readable DataFrame
ppdf=ppdf[['schedule_date', 'schedule_season', 'schedule_week', 'team_home','score_home', 'score_away','team_away','team_favorite_id', 'spread_favorite', 'results','results-spread', 'favorite_cover', 'over_under_line', 'total','over_diff', 'over_cover','weather_temperature','weather_wind_mph', 'stadium','coordinates', 'zipcode']]



In [10]:
#Test path to see if we are getting the right data
pracdf=ppdf.copy()
page = r.get('https://www.almanac.com/weather/history/zipcode/'+pracdf.loc[4450]['zipcode']+'/'+ pracdf.loc[4450]['schedule_date'])
soup= BeautifulSoup(page.content, 'html.parser')
prcp= soup.find('tr', class_= 'weatherhistory_results_datavalue prcp')
data=prcp.find('td')
nxt=data.find('p')
point=nxt.find(class_='value')
point.get_text()

u'0.49'

In [11]:

#create dic to store values
precipitation= {}
# loop through index and populate dict with precipitation data 
for i in pracdf.index:
    page = r.get('https://www.almanac.com/weather/history/zipcode/'+pracdf.loc[i]['zipcode']+'/'+ pracdf.loc[i]['schedule_date'])
    soup= BeautifulSoup(page.content, 'html.parser')
    prcp= soup.find('tr', class_= 'weatherhistory_results_datavalue prcp')
    data=prcp.find('td')
    nxt=data.find('p')
    point=nxt.find(class_='value')
    precipitation[i]=point.get_text()
    
precipitation=pd.Series(precipitation)
precipitation

AttributeError: 'NoneType' object has no attribute 'find'

In [69]:
ppdf.to_pickle('clean_data.pkl')